In [8]:
from ultralytics import YOLO
from os import path, listdir
from shutil import copytree, rmtree, copy
import time

In [9]:
# Constants
DATAPATH = './src/vision/models'
CHECKPOINT_PATH = f'{DATAPATH}/models/checkpoints/'
IMAGE_SIZE = 640 # x 480
RECIPE_NAME = "all"
TRAIN_RUN = f"{RECIPE_NAME}_run-{time.strftime('%Y%m%d-%H%M%S')}"

# Model
MODEL_PATH = f"{DATAPATH}/pretrained/yolov8n-obb-dotav1.pt"
TRAIN_PARAM = {
    # Model definition
    'data': f"{DATAPATH}/recipes/dataset_desc.yaml",
    'resume': False,
    'device': '0',
    'pretrained': True,
    # Names
    'project' : RECIPE_NAME,
    'name': TRAIN_RUN,
    # Training Parameters
    'batch': -1,
    'imgsz': IMAGE_SIZE,
    'epochs': 200,
    'patience': 10,
    'cos_lr': True,
    # Augmentation
    'hsv_h': 0.05, # Higher than default for resistor
    'hsv_s': 0.3, # Colours should not change too much
    'hsv_v': 0.2, # Colours should not change too much
    'degrees': 180, # Rotation
    'translate': 0.1, # Translation
    'scale': 0.8, # Scaling - camera is always at the same distance
    'shear': 10.0, # Shearing
    'perspective': 0.0, # Perspective
    'flipud': 0.5, # Flip up-down
    'fliplr': 0.5, # Flip left-right
    'mosaic': 0.5, # Mosaic
    'mixup': 0.0, # Mixup
    'copy_paste': 0.0, # Copy-paste
    'crop-fraction': 1.0, # Crop fraction
    # Loss weights
    'cls' : 1.0, # Class
    'box' : 4.0, # Box accuracy
    'dfl' : 1.5, # Help manage unbalanced classes
    # Post parameters
    'save': True,
    'save_period': 5,
    'plots': False,
    # Misc
    'verbose': False,
    'logging': "tensorboard",
}

In [ ]:
# Reorganise the data
PARAM = {
    'path' : "./src/vision/dataset",
    'train' : 0.7,
    'val' : 0.2,
    'test' : 0.1,
    'include' : [
        'resistor',
        'capacitor',
        'ceramic_capacitor',
        'film_capacitor',
        'inductor',
        'led',
        'wire'
    ]
}
# Remove old dataset
DATASET_FOLDER = f"{PARAM['path']}/current"
if path.exists(DATASET_FOLDER):
    for folder in listdir(DATASET_FOLDER):
        rmtree(path.join(DATASET_FOLDER, folder))

# Get all component images from all folders
images, labels = [], []
# For component folder in dataset
for folder in listdir(PARAM['path']):
    # Skip if not in include
    if folder not in PARAM['include']: continue
    # For each subfolder in component folder
    for subfolder in listdir(path.join(PARAM['path'], folder)):
        # Append all files in subfolder to images
        imgfiles = listdir(path.join(PARAM['path'], folder, subfolder, 'imgs'))
        labfiles = listdir(path.join(PARAM['path'], folder, subfolder, 'labels'))
        for file in imgfiles:
            images.append(path.join(PARAM['path'], folder, subfolder, 'imgs', file))
        for label in labfiles:
            labels.append(path.join(PARAM['path'], folder, subfolder, 'labels', label))
        print(f"Found {len(imgfiles)} images and {len(labfiles)} labels in {folder}/{subfolder}")

# Split the data into train, val, test


In [6]:
# Train
model = YOLO(MODEL_PATH)
model.train(**TRAIN_PARAM)

Model loaded on cuda


In [13]:
# Tensorboard logging
%load_ext tensorboard
%tensorboard --logdir f"{CHECKPOINT_PATH}/logs"

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6008 (pid 44968), started 0:00:04 ago. (Use '!kill 44968' to kill it.)